### Paper Analysis + Nirvana

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-q8i6hwzq52sb9o"
api_key = os.environ["RUNPOD_API_KEY"]
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
# (I). Given a paper's link, extract author & institution & citations

from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode

extract_paper_info = MetaPrompt(
    "Extract author, institution, citations, date from a paper's link.",
    "extract_paper_info",
    ["link"],
    ["authors", "institution", "citations", "date"],
    ["str"],
    ["list", "str", "list", "str"],
    PromptMode.PROMPT
)

extract_paper_info_test_cases  = [
    ({"link": "https://arxiv.org/abs/2402.15391"}, {"authors": ["Jake Bruce", "Michael Dennis", "Ashley Edwards", "Jack Parker-Holder", "Yuge Shi", "Edward Hughes", "Matthew Lai", "Aditi Mavalankar", "Richie Steigerwald", "Chris Apps", "Yusuf Aytar", "Sarah Bechtle", "Feryal Behbahani", "Stephanie Chan", "Nicolas Heess", "Lucy Gonzalez", "Simon Osindero", "Sherjil Ozair", "Scott Reed", "Jingwei Zhang", "Konrad Zolna", "Jeff Clune", "Nando de Freitas", "Satinder Singh", "Tim Rocktäschel"],
                                                    "institution": "Google DeepMind",
                                                    "citations": ["The deepmind jax ecosystem",
                                                                "Frozen in time: A joint video and image encoder for end-to-end retrieval",
                                                                "Video pretraining (vpt): Learning to act by watching unlabeled online videos",
                                                                "Neural game engine: Accurate learning of generalizable forward models from pixels",
                                                                "Human-timescale adaptation in an open-ended task space",
                                                                "Stable video diffusion: Scaling latent video diffusion models to large datasets",
                                                                "Align your latents: High-resolution video synthesis with latent diffusion models"],
                                                    "date": "2024-02-15"}),
    ({"link": "https://arxiv.org/abs/2411.18553"}, {"authors": ["Darius Feher", "Benjamin Minixhofer", "Ivan Vulić"],
                                                    "institution": "University of Cambridge",
                                                    "citations": ["Do all languages cost the same? tokenization in the era of commercial language models",
                                                                "CharacterBERT: Reconciling ELMo and BERT for word-level open-vocabulary representations from characters",
                                                                "Can Large Language Models Be an Alternative to Human Evaluations?",
                                                                "Canine: Pre-training an efficient tokenization-free encoder for language representation",
                                                                "Unsupervised Cross-lingual Representation Learning at Scale"],
                                                    "date": "2024-11-18"})
    ]


node = EvolNode(extract_paper_info, None, None, get_response=get_endpoint_response, test_cases=extract_paper_info_test_cases)
node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True)

input_dict = {"link": "https://arxiv.org/abs/2411.18553"}
output_dict = node(input_dict)
print("Output dict: ", output_dict)


In [8]:
reasonings, codes = node._evolve("i1", batch_size=20)

max_tries = 3
num_runs = 2
timeout = True
self = node 
# codes = node.codes 

# fitness_per_code, errors_per_code, global_summary = self._evaluate_fitness(codes=codes, max_tries=max_tries, num_runs=num_runs, custom_metric_map=self.custom_metric_map, timeout=timeout)


test_cases = node.test_cases
test_inputs = [case[0] for case in test_cases]

output_per_code_per_test, errors_per_code_per_test = node.call_prompt_function_parallel(test_inputs, codes, max_tries)

print("Output per code per test: ", output_per_code_per_test)
print("Errors per code per test: ", errors_per_code_per_test)

Processing LLM queries: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]


 :: Total time elapsed: 26.46s, 0 errors
ERROR PARSING CODE
ERROR PARSING CODE


Concurrent Execution of 36 Node functions to generate prompts ...: 100%|██████████| 36/36 [00:02<00:00, 17.71it/s]
Processing LLM queries: 100%|██████████| 24/24 [00:22<00:00,  1.05it/s]

 :: Total time elapsed: 22.84s, 0 errors
Output per code per test:  defaultdict(<function call_func_prompt_parallel.<locals>.<lambda> at 0x2865c8900>, {15: defaultdict(<class 'dict'>, {1: {'authors': ['Yuxiang Hou, Yuanyuan Shi, Zhihao Jiao, Xiao Sun, Jianwei Liang, Depeng Li'], 'institution': 'University of Science and Technology of China', 'citations': ['2'], 'date': '2022-11-07'}}), 5: defaultdict(<class 'dict'>, {1: {'authors': [{'name': "{author['name']}", 'affiliation': "{author['affiliation']}", 'email': "{author['email']}"}], 'institution': {'name': "{institution['name']}", 'address': "{institution['address']}"}, 'citations': [{'title': "{citation['title']}", 'year': "{citation['year']}", 'journal': "{citation['journal']}", 'volume': "{citation['volume']}", 'pages': "{citation['pages']}"}], 'date': '{date}'}, 0: {'authors': [{'name': "{author['name']}", 'affiliation': "{author['affiliation']}"}], 'institution': {'name': "{institution['name']}", 'location': "{institution['locati

In [8]:
from collections import defaultdict
from methods.meta_execute import process_all_inputs
from methods.evolnode import extract_json_from_text

get_response = node.get_response
input_dicts = test_inputs

outputs_per_code_per_test = defaultdict(lambda: defaultdict(list))
errors_per_code_per_test = defaultdict(lambda: defaultdict(list))

prompts = []
input_indices = []

# Multi-thread execution (I love GPU parallelism better now ... this is just frustratingly slow)    
input_indices, full_prompts, errors_per_code_per_test = process_all_inputs(codes, input_dicts, max_tries)
        
desc_str = f"Executing prompt node with LLM in parallel with batch size {len(prompts)}"
responses = get_response(full_prompts, desc=desc_str)

for (response, (input_index, code_index)) in zip(responses, input_indices):
    try:
        output_dict = extract_json_from_text(response)
        outputs_per_code_per_test[code_index][input_index].append(output_dict)
    
    except Exception as e:
        errors_per_code_per_test[code_index][input_index].append(f"Failed to parse LLM response -- " + str(e))

Concurrent Execution of 32 Node functions to generate prompts ...:  81%|████████▏ | 26/32 [00:02<00:00, 12.56it/s]
100%|██████████| 950k/950k [00:00<00:00, 12.9MiB/s]
Concurrent Execution of 32 Node functions to generate prompts ...:  97%|█████████▋| 31/32 [00:02<00:00, 15.77it/s]






100%|██████████| 42.3M/42.3M [00:00<00:00, 43.9MiB/s]
Concurrent Execution of 32 Node functions to generate prompts ...: 100%|██████████| 32/32 [00:03<00:00, 10.25it/s]
Processing LLM queries: 100%|██████████| 24/24 [00:24<00:00,  1.03s/it]

 :: Total time elapsed: 24.70s, 0 errors


In [10]:
# input_indices
responses

['To achieve the task, you\'ll need to install the `requests` library to make HTTP requests to the arXiv API and the `json` library to manipulate JSON data. I\'ll provide the code to perform the task.\n\n```python\nimport requests\nimport json\n\ndef get_paper_metadata(paper_link):\n    """\n    Get paper metadata from arXiv API.\n    \n    Args:\n    - paper_link (str): arXiv paper link.\n    \n    Returns:\n    - metadata (dict): Paper metadata.\n    """\n    api_url = f"https://arxiv.org/api/query?id_list={paper_link}"\n    headers = {\'accept\': \'application/xml\'}\n    response = requests.get(api_url, headers=headers)\n\n    if response.status_code == 200:\n        metadata = {}\n        for entry in response.from_xml().findall(\'.//entry\'):\n            metadata[\'title\'] = entry.find(\'title\').text\n            metadata[\'authors\'] = [author.text for author in entry.findall(\'author\')]\n            metadata[\'institution\'] = entry.find(\'author\').attrib.get(\'affiliation

In [6]:
node.codes

['import arxiv\nfrom arxiv import Client, Search, Result\nimport brotli\nfrom concurrent.futures import ThreadPoolExecutor\nfrom datetime import datetime\nimport gzip\nimport os\nimport requests\nimport time\nfrom tqdm import tqdm\nfrom typing import Tuple\nfrom typing import List, Dict\nfrom typing import List, Dict, Any\nimport zlib\n\n\ndef download_large_file(url: str, output_path: str, chunk_size: int=8192,\n    timeout: int=30) ->Tuple[bool, int, float, str]:\n    try:\n        start_time = time.time()\n        headers = {\'User-Agent\': \'Mozilla/5.0\'}\n        head_response = requests.head(url, headers=headers, timeout=timeout)\n        file_size = int(head_response.headers.get(\'content-length\', 0))\n        content_encoding = head_response.headers.get(\'content-encoding\',\n            \'identity\')\n        progress = tqdm(total=file_size, unit=\'iB\', unit_scale=True)\n        with requests.get(url, headers=headers, stream=True, timeout=timeout\n            ) as response:

In [ ]:
# present the workflow with front-end UI & show information flow 